In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.llms.lmstudio import LMStudio
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent

llm = LMStudio(
    model_name="TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q2_K.gguf",
    base_url="http://localhost:1234/v1",
    temperature=0.7,
)

In [3]:
# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [4]:
def generate_hypotheses(problem:str) -> str:
    """Given a problem and some context, generate a few hypotheses for what the solution is"""
    prompt = """
        You are an expert at problem solving. You take in a problem and you come up with a few hypotheses for what is causing the problem. 
        For example you might be given:
            Problem: I can't log into my email
            Hypotheses: ['Password was recently changed.' 'Wifi might not be connected']
        
        Now, for the new problem.
        Problem:
    """
    results = llm.stream_complete(prompt)
    return results

hypotheses_tool = FunctionTool.from_defaults(fn=generate_hypotheses)

In [5]:
agent = ReActAgent.from_tools([multiply_tool,hypotheses_tool], llm=llm, verbose=True)

In [6]:
agent.chat("Why did my car break down?")

Thought: The current question is in English, and I need to use a tool or multiple tools to help me answer the question. However, at this point, I don't have enough information from the user or the context of the conversation to determine which tool(s) would be best to use.

Therefore, I will first generate some hypotheses about what might have caused the car to break down based on general knowledge and common car-related problems. Then, if the hypotheses are not sufficient to answer the question, I may need to request more context from the user or explore specific tools further.
Action: generate_hypotheses
Action Input: {'problem': 'Why did my car break down?'}
Observation: <generator object llm_completion_callback.<locals>.wrap.<locals>.wrapped_llm_predict.<locals>.wrapped_gen at 0x3036ced50>
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Thought: I received an error message from the `generate_hypotheses` tool. It appears that the

AgentChatResponse(response="I'm unable to definitively determine why your car broke down without more information. Could you please provide some context, such as the make, model, symptoms, or any error codes you may have seen on the dashboard? This will help narrow down the possibilities and potentially give a more accurate answer.", sources=[ToolOutput(content='<generator object llm_completion_callback.<locals>.wrap.<locals>.wrapped_llm_predict.<locals>.wrapped_gen at 0x3036ced50>', tool_name='generate_hypotheses', raw_input={'args': (), 'kwargs': {'problem': 'Why did my car break down?'}}, raw_output=<generator object llm_completion_callback.<locals>.wrap.<locals>.wrapped_llm_predict.<locals>.wrapped_gen at 0x3036ced50>, is_error=False), ToolOutput(content='Error: Could not parse output. Please follow the thought-action-input format. Try again.', tool_name='unknown', raw_input={}, raw_output='Error: Could not parse output. Please follow the thought-action-input format. Try again.', i